# Checking and describing the generated data

It is always beneficial to add a notebook that quickly looks into the data to help you remember, which data you collected and if it actually looks correct.

In [20]:
from algbench import describe, read_as_pandas, Benchmark
from _conf import EXPERIMENT_DATA

In [21]:
describe(EXPERIMENT_DATA)

An entry in the database can look like this:
_____________________________________________
 result:
| num_nodes: 25
| lower_bound: 118462718.0
| objective: 118462718.0
 timestamp: 2023-11-16T09:40:28.274127
 runtime: 0.01595783233642578
 stdout: [[0.0005774497985839844, 'Set parameter Username\n'], [0.0013246536254882812,...
 stderr: []
 logging: []
 env_fingerprint: fb673f25c8f859480497400028c888d1f4d2a473
 args_fingerprint: 00e90df2fbbd823b698220e9ecb0834d856a16cf
 parameters:
| func: run_solver
| args:
|| instance_name: random_euclidean_25_0
|| time_limit: 90
|| strategy: GurobiTspSolver
 argv: ['01_run_benchmark.py']
 env:
| hostname: workstation-r7
| python_version: 3.10.12 (main, Jul  5 2023, 18:54:27) [GCC 11.2.0]
| python: /home/krupke/anaconda3/envs/mo310/bin/python3
| cwd: /home/krupke/Repositories/cpsat-primer/examples/tsp_evaluation
| git_revision: 1e62bceecf6ef56b59edca6f4ee2ae652da012b5
| python_file: /home/krupke/Repositories/cpsat-primer/examples/tsp_evaluation/01_run_b

In [22]:
t = read_as_pandas(
    EXPERIMENT_DATA,
    lambda entry: {
        "instance_name": entry["parameters"]["args"]["instance_name"],
        "num_nodes": entry["result"]["num_nodes"],
        "time_limit": entry["parameters"]["args"]["time_limit"],
        "strategy": entry["parameters"]["args"]["strategy"],
        "runtime": entry["runtime"],
        "objective": entry["result"]["objective"],
        "lower_bound": entry["result"]["lower_bound"],
    },
)
t.drop_duplicates(inplace=True, subset=["instance_name", "num_nodes", "strategy"])
t["opt_gap"] = (t["objective"] - t["lower_bound"]) / t["lower_bound"]
t.sort_values(["num_nodes", "instance_name"])

,instance_name,num_nodes,time_limit,strategy,runtime,objective,lower_bound,opt_gap
0,random_euclidean_25_0,25,90,GurobiTspSolver,0.015958,118462718.0,1.184627e+08,0.000000
1,random_euclidean_25_0,25,90,CpSatTspSolverV1,0.060843,118462718.0,1.184627e+08,0.000000
490,random_euclidean_25_0,25,90,CpSatTspSolverMtz,0.127046,118462718.0,1.184627e+08,0.000000
527,random_euclidean_25_0,25,90,CpSatTspSolverDantzig,0.204676,118462718.0,1.184627e+08,0.000000
2,random_euclidean_25_1,25,90,GurobiTspSolver,0.014089,80173028.0,8.017303e+07,0.000000
...,...,...,...,...,...,...,...,...
485,random_euclidean_400_7,400,90,CpSatTspSolverV1,95.111182,316619716.0,6.574902e+07,3.815581
486,random_euclidean_400_8,400,90,GurobiTspSolver,90.999890,92159974.0,7.090806e+07,0.299711
487,random_euclidean_400_8,400,90,CpSatTspSolverV1,95.360385,254279820.0,6.557753e+07,2.877545
488,random_euclidean_400_9,400,90,GurobiTspSolver,53.188244,71300824.0,7.130082e+07,0.000000


In [23]:
for entry in Benchmark(EXPERIMENT_DATA):
    
    if entry["parameters"]["args"]["instance_name"] == "random_euclidean_100_1" and entry["parameters"]["args"]["strategy"] == "GurobiTspSolver":
        print("=====================================")
        stdout = "".join( e[1] for e in entry["stdout"])
        stderr = "".join( e[1] for e in entry["stderr"])
        print(stdout)
        print(stderr)
        if not stdout.strip():
            print("No stdout")
        print("=====================================")

Set parameter TimeLimit to value 90
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 100 rows, 4950 columns and 9900 nonzeros
Model fingerprint: 0x6dd941da
Variable types: 0 continuous, 4950 integer (4950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 100 rows, 4950 columns, 9900 nonzeros
Variable types: 0 continuous, 4950 integer (4950 binary)

Root relaxation: objective 6.947760e+07, 108 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6.9478e+07    

## Check for errors in the data

You always want to check if the results you got are actually feasible. Errors easily happen and are not always visible on the plots.
Thus, you want to do some basic checks to detect errors early on. For example, you could accidentally have swapped lower and upper bounds in the data generation process.
Depending on your plots, this may not be visible, and you may end up comparing the wrong data and draw the wrong conclusions.
Or, you could have accidentally swapped runtime and objective values, which could look reasonable in the data as the runtime and the objective often increase with the instance size.

A very basic check is to check if the best lower and upper bounds do not contradict each other. Many errors will be caught by this check. However, you often need some tolerance to account for numerical errors.

In [24]:
assert (t.dropna()["opt_gap"]>=-0.0001).all(), "Optimality gap is negative!"

In [25]:
# Always make sure that your results are not trivially wrong
#  - e.g. lower bound is higher than objective
max_lb = t.groupby(["instance_name"])["lower_bound"].max()
min_obj = t.groupby(["instance_name"])["objective"].min()
eps = 0.0001  # some tolerance is needed when working with floats.
bad_instances = max_lb[max_lb-min_obj>eps*max_lb].index.to_list()
assert len(bad_instances) == 0, "Bad instances detected: {}".format(bad_instances)
# t[t["instance_name"].isin(bad_instances)]